# Depencency parsing
(парсинг зависимостей)

## Что это?

* наша цель -- представить предложение естественного языка в виде дерева
* слова предложения -- вершины; *зависимости (dependencies)* между ними -- рёбра
* зависимости могут быть разными: например, субъект глагола, объект глагола, прилагательное-модификатор, и так далее

## Формат

Существует несколько форматов записи деревьев зависимостей, но самый популярный и общеиспользуемый -- [CoNLL-U](http://universaldependencies.org/format.html).<br/>
Как это выглядит (пример из [русского Universal Dependency трибанка](https://github.com/UniversalDependencies/UD_Russian-SynTagRus)):

In [26]:
my_example = """
# sent_id = 2003Armeniya.xml_138
# text = Перспективы развития сферы высоких технологий.
1	Перспективы	перспектива	NOUN	_	Animacy=Inan|Case=Nom|Gender=Fem|Number=Plur	0	root	0:root	_
2	развития	развитие	NOUN	_	Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing	1	nmod	1:nmod	_
3	сферы	сфера	NOUN	_	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing	2	nmod	2:nmod	_
4	высоких	высокий	ADJ	_	Case=Gen|Degree=Pos|Number=Plur	5	amod	5:amod	_
5	технологий	технология	NOUN	_	Animacy=Inan|Case=Gen|Gender=Fem|Number=Plur	3	nmod	3:nmod	SpaceAfter=No
6	.	.	PUNCT	_	_	1	punct	1:punct	_
"""

Комментарии + таблица c 9 колонками (разделители табы):
* ID
* FORM: токен
* LEMMA: начальная форма
* UPOS: универсальная часть речи
* XPOS: лингво-специфичная часть речи
* FEATS: морфологическая информация: падеж, род, число etc
* HEAD: id ролителя
* DEPREL: тип зависимости, то есть отношение к токену-родителю
* DEPS: альтернативный подграф (не будем углубляться :))
* MISC: всё остальное

Отсутствующие данные представляются с помощью `_`. Больше подробностей про формат -- в [официальной документаци](http://universaldependencies.org/format.html).<br>
User-friendly визуализация: ![2003Armeniya.xml_138](rus_tree.png)

Отрытый инструмент для визуализации, ручной разметки и конвертации в другие форматы: UD Annotatrix. [Online-интерфейс](https://maryszmary.github.io/ud-annotatrix/standalone/annotator.html), [репозиторий](https://github.com/jonorthwash/ud-annotatrix).

Трибанк -- много таких предложений. Обычно они разделяются двумя переносами строки.
### Как считывать данные в питоне

Используем библиотеку [conllu](https://github.com/EmilStenstrom/conllu).

In [66]:
!pip3 install conllu
from conllu import parse

In [67]:
help(parse)

Help on function parse in module conllu.parser:

parse(text, fields=('id', 'form', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc'))



In [68]:
sentences = parse(my_example)
sentence = sentences[0]
sentence[0]

OrderedDict([('id', 1),
             ('form', 'Перспективы'),
             ('lemma', 'перспектива'),
             ('upostag', 'NOUN'),
             ('xpostag', None),
             ('feats',
              OrderedDict([('Animacy', 'Inan'),
                           ('Case', 'Nom'),
                           ('Gender', 'Fem'),
                           ('Number', 'Plur')])),
             ('head', 0),
             ('deprel', 'root'),
             ('deps', [('root', 0)]),
             ('misc', None)])

In [61]:
sentence[-1]

OrderedDict([('id', 6),
             ('form', '.'),
             ('lemma', '.'),
             ('upostag', 'PUNCT'),
             ('xpostag', None),
             ('feats', None),
             ('head', 1),
             ('deprel', 'punct'),
             ('deps', [('punct', 1)]),
             ('misc', None)])

## UDPipe

Есть разные инструменты для парсинга зависимостей. Сегодня мы будем рабтать с [UDPipe](http://ufal.mff.cuni.cz/udpipe). UDPipe умеет парсить текст с помощью готовых моделей (которые можно скачать с ...) и обучать модели на своих трибанках.

Собственно, в UDPipe есть три вида моделей:
* токенизатор (разделить предложение не токены, сделать заготовку для CoNLL-U)
* тэггер (разметить части речи)
* сам парсер (проставить каждому токену `head` и `deprel`)

Мы сегодня не будем обучать новых моделей (это слишком долго), а используем готовую модель для русского.

### The Python binding

У udpipe есть питоновская обвязка. Она довольно [плохо задокументирована](https://pypi.org/project/ufal.udpipe/), но зато можно использовать прямо в питоне :)

In [70]:
!pip3 install ufal.udpipe
from ufal.udpipe import Model, Pipeline

In [52]:
model = Model.load("/home/maryszmary/Documents/cross-lingual-parsing/utils/models/combined_11K") # path to the model

In [54]:
# если успех, должно быть так
model is None

False

In [74]:
ppln = Pipeline(model, 'generic_tokenizer', '', '', '')
far = "Í 1957 verður Útvarp Føroya stovnað."
print(ppln.process(far))

# newdoc
# newpar
# sent_id = 1
# text = Í 1957 verður Útvarp Føroya stovnað.
1	Í	_	ADP	_	_	3	case	_	_
2	1957	1957	NUM	_	_	3	nummod	_	_
3	verður	_	NOUN	_	_	4	nmod	_	_
4	Útvarp	_	PROPN	_	_	6	nsubj	_	_
5	Føroya	_	PROPN	_	_	4	flat:name	_	_
6	stovnað	_	VERB	_	_	0	root	_	SpaceAfter=No
7	.	_	PUNCT	_	_	6	punct	_	SpacesAfter=\n




### Command line interface

Но с обвязкой бывают проблемы, и вообще довольно удобно пользоваться прекомпилированной утилитой `udpipe` из шелла. Она скачивается здесь: https://github.com/ufal/udpipe/releases/tag/v1.2.0 (надеюсь, вы уже скачали всё заранее).<br>Внутри бинарники для всех популярных ОС, выбираем свою. У меня путь к бинарнику такой: `udpipe-1.2.0-bin/bin-linux64/udpipe`.

## Главред

Главред -- [сервис](https://glvrd.ru/) для корекции стиля текста. У него даже есть [API]()!<br>
На этом семинаре мы имплементируем часть его синтаксических функций.

А теперь напишите кастомную функцию, которая 